In [4]:
!pip install obspy
!pip install sourcespec
!pip install matplotlib==3.8.3
!pip uninstall scipy -y
!pip install scipy==1.10.1
!pip install sourcespec


Found existing installation: scipy 1.10.1
Uninstalling scipy-1.10.1:
  Successfully uninstalled scipy-1.10.1
  Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (58 kB)
Using cached scipy-1.10.1-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (34.1 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
cvxpy 1.6.6 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
tsfresh 0.21.0 requires scipy>=1.14.0; python_version >= "3.10", but you have scipy 1.10.1 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
xarray-einstats 0.9.1 requires scipy>=1.11, but you have

In [2]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import obspy
import h5py
from obspy import UTCDateTime
import numpy as np
from obspy.clients.fdsn.client import Client
import matplotlib.pyplot as plt
from obspy.core.utcdatetime import UTCDateTime
from obspy.core.event import Catalog, Event, Origin, Magnitude
import re
import shutil
import os
import torch
import subprocess
from concurrent.futures import ProcessPoolExecutor, as_completed
from multiprocessing import shared_memory
import time

In [3]:
def make_stream(dataset):
    '''
    input: hdf5 dataset
    output: obspy stream

    '''
    data = np.array(dataset)

    tr_E = obspy.Trace(data=data[:, 0])
    tr_E.stats.starttime = UTCDateTime(dataset.attrs['trace_start_time'])
    tr_E.stats.delta = 0.01
    if(dataset.attrs['network_code'] == 'PB'):
        tr_E.stats.channel = dataset.attrs['receiver_type']+'1'
    else:
        tr_E.stats.channel = dataset.attrs['receiver_type']+'E'
    tr_E.stats.station = dataset.attrs['receiver_code']
    tr_E.stats.network = dataset.attrs['network_code']
    if(dataset.attrs['network_code'] == 'GM'):
        tr_E.stats.location = '01'
    elif(dataset.attrs['network_code'] in ['II', 'US', 'NM', 'ET']):
        tr_E.stats.location = '00'

    tr_N = obspy.Trace(data=data[:, 1])
    tr_N.stats.starttime = UTCDateTime(dataset.attrs['trace_start_time'])
    tr_N.stats.delta = 0.01
    if(dataset.attrs['network_code'] == 'PB'):
        tr_N.stats.channel = dataset.attrs['receiver_type']+'2'
    else:
        tr_N.stats.channel = dataset.attrs['receiver_type']+'N'
    tr_N.stats.station = dataset.attrs['receiver_code']
    tr_N.stats.network = dataset.attrs['network_code']
    if(dataset.attrs['network_code'] == 'GM'):
        tr_N.stats.location = '01'
    elif(dataset.attrs['network_code'] in ['II', 'US', 'NM', 'ET']):
        tr_N.stats.location = '00'

    tr_Z = obspy.Trace(data=data[:, 2])
    tr_Z.stats.starttime = UTCDateTime(dataset.attrs['trace_start_time'])
    tr_Z.stats.delta = 0.01
    tr_Z.stats.channel = dataset.attrs['receiver_type']+'Z'
    tr_Z.stats.station = dataset.attrs['receiver_code']
    tr_Z.stats.network = dataset.attrs['network_code']
    if(dataset.attrs['network_code'] == 'GM'):
        tr_Z.stats.location = '01'
    elif(dataset.attrs['network_code'] in ['II', 'US', 'NM', 'ET']):
        tr_Z.stats.location = '00'

    stream = obspy.Stream([tr_E, tr_N, tr_Z])

    return stream



file_name = "STEAD_data/chunk2/chunk2.hdf5"
csv_file = "STEAD_data/chunk2/chunk2.csv"

# reading the csv file into a dataframe:
df = pd.read_csv(csv_file)
print(f'total events in csv file: {len(df)}')
#df_filtered = df[(df.trace_category == 'earthquake_local') & (df.source_distance_km <= 10) & (df.source_magnitude > 4)]
df_filtered = df[~df.network_code.isin(['IV', 'HA', 'KO', 'HP', 'FR', 'S', 'TU'])]
print(f'total events selected: {len(df_filtered)}')


dtfl = h5py.File(file_name, 'r')
ev_list = df_filtered['trace_name'].to_list()

client = Client("IRIS")

num_events = len(ev_list[0:10000])
fc_array = torch.empty((num_events, 1), dtype=torch.float32)
fc_array.fill_(torch.nan)


for c, evi in enumerate(ev_list[0:10000]):
        print(f"\nITERATION {c}\n")
        dataset = dtfl.get('data/'+str(evi))
        print(f'\n{str(evi)}\n')

        inventory = None
        attempt = 0
        max_attempts = 5
        while attempt < max_attempts:
            try:
                inventory = client.get_stations(
                    network=dataset.attrs['network_code'],
                    station=dataset.attrs['receiver_code'],
                    starttime=UTCDateTime(dataset.attrs['trace_start_time']),
                    endtime=UTCDateTime(dataset.attrs['trace_start_time']) + 60,
                    loc="*",
                    channel="*",
                    level="response"
                )
                break

            except Exception as e:
                attempt += 1
                print(f"Try {attempt}/{max_attempts} failed: {e}")
                time.sleep(10)

        if inventory is None:
            print("Error: impossible to connect to server after numerous tries.")
            continue


        # converting into acceleration
        st = make_stream(dataset)
        try:
          st = st.remove_response(inventory=inventory, output="ACC", plot=False)
        except Exception as e:
          print(f"Stream-wide response removal failed: {e}")
          continue

        event = Event()
        origin = Origin(
            time=UTCDateTime(dataset.attrs['trace_start_time']),
            latitude=dataset.attrs.get('source_latitude', 45.123),
            longitude=dataset.attrs.get('source_longitude', 7.456),
            depth=dataset.attrs.get('source_depth', 10000)
        )
        magnitude = Magnitude(mag=dataset.attrs['source_magnitude'])
        event.origins.append(origin)
        event.magnitudes.append(magnitude)
        catalog = Catalog(events=[event])
        event_id = catalog[0].resource_id.id
        catalog.write("event.xml", format="QUAKEML")
        inventory.write("stations.xml", format="STATIONXML")
        st.write("event.mseed", format="MSEED")

        try:
            result = subprocess.run(
                ["source_spec", "-c", "source_spec.conf", "-t", "event.mseed", "-q", "event.xml", "-o", "output_dir"],
                capture_output=True,  # To capture both the output and the error
                text=True,  # To get the output as strings instead of bytes
                check=True  # Raises an exception if the command returns an error
            )
            print("Command executed successfully!")
            print("Output:")
            print(result.stdout)
        except subprocess.CalledProcessError as e:
            print(f"Error executing the command: {e}")
            print(f"Error message: {e.stderr}")
            continue

        event_id= event_id[10:]

        file_path=f"output_dir2/{event_id}/{event_id}.ssp.out"

        if os.path.exists(file_path):
          with open(file_path, "r") as file:
            content = file.readlines()
          fc_value = torch.nan
          for line in content:
            match = re.search(r"fc\s+([\d.]+)", line)
            if match:
                fc_value = float(match.group(1))
                fc_array[c]=fc_value
                break
        else:
            print(f"{event_id}.ssp.out file not created")

        folder_to_remove = os.path.join("output_dir", event_id)
        if os.path.exists(folder_to_remove) and os.path.isdir(folder_to_remove):
            shutil.rmtree(folder_to_remove)
            print(f"Folder '{folder_to_remove}' removed!")
        else:
            print(f"Folder '{folder_to_remove}' doesn't exist or it's not a directory.")

print(fc_array[:10])


FileNotFoundError: [Errno 2] No such file or directory: 'chunk2/chunk2.csv'